# Welkom bij de onderzoeks pipeline voor het vak Data Science.
----

### Dit onderzoek is opgezet door groep 6 van klas V2C:
- Sebastiaan Jansen
- Mustafa Toprak
- Skott de Koster

#### Dit data onderzoek is gebaseerd op de dataset over films. Het bevat een aantal onderzoeksvragen die wij zullen behandelen:
- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
- Tot hoeverre variëren  de genres van de films waarin een acteur speelt verspreid over zijn/haar carrière?
- Hoeveel effect heeft de lengte van een trailer op de omzet van de film?
- Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op facebook?
- Welke regisseurs boeken de meeste winst?
- Wat is de totale winst van alle films per jaar?
 
Deze vragen zullen allemaal worden behandeld in dit onderzoek.

## Stap 1: Data collection

De dataset is aan ons gegeven door de Hogeschool Utrecht als onderdeel van de opdracht. Deze dataset houd een groot csv bestand in met onder andere de film titel, genre en een paar belangrijke acteurs. We hebben dus nu de data verzameld en gaan het nu inlezen.

---

## Stap 2: Data processing

We gaan nu de dataset inlezen en verwerken om het beter te kunnen bekijken. Om te beginnen importeren we de benodigde Python libraries:

In [ ]:
# Deze libraries zijn voor het verwerken van de data.
import numpy as np
import pandas as pd

# Deze libraries zijn voor het verkrijgen van YouTube video gegevens, dit komt later aan bod.
import pafy
import youtube_dl
import datetime 

Vervolgens lezen we de eerste (toegewezen) dataset in. Hiervoor gebruiken we pandas.read_csv en slaan we dit op in de dataframe "films". 
Aangezien de dataset erg groot is, laten we voor nu alleen de eerste 5 rijen zien. Dit doen we door een variabele "films_toegewezen" aan te maken en de dataset op te slaan als dataframe:

In [ ]:
films_toegewezen = pd.read_csv('movie.csv', sep=",")
films_toegewezen.head()

Hetzelfde doen we met de tweede (externe) dataset, deze noemen we "films_extern": 

In [ ]:
films_extern = pd.read_csv('ml-youtube.csv', sep=",")
films_extern.head()

We zien dus dat de eerste dataset bestaat uit 5043 films, dit moeten we nu combineren met de tweede dataset. Maar de tweede dataset heeft veel meer rows en een iets andere vorm van filmtitel. Dit moeten we dus oplossen, en ook de andere problemen zoals de NaN en duplicate waarden.

---

# Stap 3: Data cleaning

We gaan nu de data schoon maken. Om te beginnen schonen we de eerste dataset op door alle data die niet relevant is te verwijderen:

In [ ]:
to_drop = ['color','facenumber_in_poster','country','aspect_ratio','language','content_rating','plot_keywords']
films_toegewezen.drop(to_drop, inplace=True, axis=1)

films_extern.drop('movieId', inplace=True, axis=1)

In [ ]:
# We verwijderen de rijen met NaN gegevens in beide datasets, aangezien dit data is die we niet kunnen gebruiken.
films_toegewezen = films_toegewezen.dropna()
films_extern = films_extern.dropna()

In [ ]:
# We halen alle duplicates eruit.
films_toegewezen = films_toegewezen.drop_duplicates(subset="movie_title")
films_extern = films_extern.drop_duplicates(subset="title")

In [ ]:
# We slicen de laatste 7 characters van de title column (het jaar dat de film uitkwam) in de tweede dataset weg,
# zodat de column overeen komt met de column in de eerste dataset.
# Ook halen we een whitespace weg in de eerste dataset die er niet in hoort te zitten.
films_toegewezen.update(films_toegewezen['movie_title'].str[:-1])
films_extern.update(films_extern['title'].str[:-7])

In [ ]:
# We setten de indexes van beide datasets naar de titel van de films.
films_toegewezen.set_index('movie_title', inplace=True)
films_extern.set_index('title', inplace=True)

In [ ]:
# We sorteren beide datasets zodat de waardes gelijk zullen zijn.
films_toegewezen = films_toegewezen.sort_index()
films_extern = films_extern.sort_index()

Nu ziet de eerste dataset er zo uit:

In [ ]:
films_toegewezen.head()

En ziet de tweede dataset er zo uit:

In [ ]:
films_extern.head()

Nu gaan we beide datasets samenvoegen tot 1 dataframe, deze noemen we 'films'. Het idee is om de youtubeId column van de tweede dataset te verwerken in de eerste dataset.

In [ ]:
# Omdat de youtubeId column het enige is wat relevant is, is dat ook het enige veld dat we toevoegen. 
# De indexes van de eerste en tweede dataset worden samengevoegd tot 1 index genaamd title.
films = pd.merge(films_toegewezen, films_extern, left_index=True, right_index=True)
films.index.name = 'title'
films

Je kunt zien dat er 2837 films zijn. Dit is bijna de helft minder dan er in de oorspronkelijke eerste dataset stond, dit is omdat we de onbruikbare data hebben verwijderd en er alleen bruikbare data is overgebleven.
We gaan de gemergede dataset nu verkennen en analyseren.

---